<a href="https://colab.research.google.com/github/claudio1975/FAC_IA_Attuari/blob/main/FAC_IA_Attuari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Importazione delle librerie

In [ ]:
!pip install -U langchain-openai langchain &>/dev/null

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "***********************"

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import random
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from IPython.display import display, Markdown

### 2. Connessione con SQLite Database

In [ ]:
# Connessione a SQLite database
conn = sqlite3.connect('attuari.db')
cursor = conn.cursor()

### 3. Creazione delle tabelle

In [ ]:
cursor.execute('''
DROP TABLE IF EXISTS polizze;
''')
# Creazione tabella polizze
cursor.execute('''
CREATE TABLE polizze (
    numero_polizza INTEGER PRIMARY KEY,
    prodotto TEXT,
    data_decorrenza DATE,
    data_scadenza DATE
);
''')

cursor.execute('''
DROP TABLE IF EXISTS riserve;
''')
# Creazione tabella riserve
cursor.execute('''
CREATE TABLE riserve (
    numero_polizza INTEGER PRIMARY KEY,
    importo_riserva NUMERIC,
    data_riserva DATE
);
''')

conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM polizze;", conn)

,numero_polizza,prodotto,data_decorrenza,data_scadenza


In [ ]:
pd.read_sql_query("SELECT * FROM riserve;", conn)

,numero_polizza,importo_riserva,data_riserva


### 4. Generazione e inserimento dei dati

In [ ]:
random.seed(0)

# generazione di 100 polizze
numero_polizza = list(range(1000, 1100))

# Creazione del dataframe polizze
prodotti = ["CAPITALE","MISTA"]
data_iniziale = datetime(2020, 1, 1)
polizza = []
for i in numero_polizza:
    prodotto = random.choice(prodotti)
    data_decorrenza = data_iniziale + timedelta(days=random.randint(0, 90))
    data_scadenza = data_decorrenza + timedelta(days=365*8)
    polizza.append((i, prodotto, data_decorrenza.date(), data_scadenza.date()))

# Inserimento dei dati in tabella polizze
cursor.executemany('''
INSERT INTO polizze (numero_polizza, prodotto, data_decorrenza, data_scadenza)
VALUES (?, ?, ?, ?)
''', polizza)
conn.commit()



In [ ]:
pd.read_sql_query("SELECT *  FROM polizze;", conn)

,numero_polizza,prodotto,data_decorrenza,data_scadenza
0,1000,MISTA,2020-02-23,2028-02-21
1,1001,CAPITALE,2020-02-03,2028-02-01
2,1002,MISTA,2020-02-21,2028-02-19
3,1003,MISTA,2020-03-02,2028-02-29
4,1004,MISTA,2020-03-15,2028-03-13
...,...,...,...,...
95,1095,CAPITALE,2020-02-28,2028-02-26
96,1096,CAPITALE,2020-02-12,2028-02-10
97,1097,CAPITALE,2020-03-10,2028-03-08
98,1098,MISTA,2020-01-18,2028-01-16


In [ ]:
# Creazione dataframe riserve
riserva = []
for i in numero_polizza:
  importo_riserva = round(random.uniform(1000.00, 100000.00), 2)
  data_riserva = data_iniziale + timedelta(days=365*5)
  riserva.append((i, importo_riserva, data_riserva.date()))

# Inserimento dei dati in tabella riserve
cursor.executemany('''
INSERT INTO riserve (numero_polizza, importo_riserva, data_riserva)
VALUES (?, ?, ?)
''', riserva)
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM riserve", conn)

,numero_polizza,importo_riserva,data_riserva
0,1000,35870.33,2024-12-30
1,1001,29499.91,2024-12-30
2,1002,36560.92,2024-12-30
3,1003,94743.68,2024-12-30
4,1004,63741.04,2024-12-30
...,...,...,...
95,1095,34487.34,2024-12-30
96,1096,62433.27,2024-12-30
97,1097,5079.09,2024-12-30
98,1098,17222.19,2024-12-30


# Prompt Engineering per la programmazione

### Utilizzo di LangChain e GPT-4o-mini per creare una query di collegamento tra tabelle

In [ ]:
# Definizione del prompt
prompt = PromptTemplate(
    input_variables=["descrizione"],
    template="""
Sei un esperto analista in SQL. Scrivi una query in SQL per collegare le due tabelle secondo la descrizione fornita.

Tabelle:
1. polizze (numero_polizza INTEGER PRIMARY KEY, prodotto TEXT, data_decorrenza DATE, data_scadenza DATE)
2. riserve (numero_polizza INTEGER PRIMARY KEY, importo_riserva NUMERIC, data_riserva DATE)

Regole:
{descrizione}

SQL Query:
"""
)

chat = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
chain = prompt | chat
regole = "Recupera tutte le polizze con tutti i dati dalle tabelle."

sql_query = chain.invoke({"descrizione": regole})

display(Markdown(sql_query.content))

Per collegare le due tabelle `polizze` e `riserve` e recuperare tutte le polizze con i relativi dati, puoi utilizzare una query SQL con un `LEFT JOIN`. Questo ti permetterà di ottenere tutte le righe dalla tabella `polizze` e, se disponibili, i dati corrispondenti dalla tabella `riserve`. Ecco la query SQL:

```sql
SELECT 
    p.numero_polizza,
    p.prodotto,
    p.data_decorrenza,
    p.data_scadenza,
    r.importo_riserva,
    r.data_riserva
FROM 
    polizze p
LEFT JOIN 
    riserve r ON p.numero_polizza = r.numero_polizza;
```

In questa query:
- `p` è un alias per la tabella `polizze`.
- `r` è un alias per la tabella `riserve`.
- La clausola `LEFT JOIN` assicura che tutte le polizze siano incluse nel risultato, anche se non ci sono riserve associate. Se non ci sono riserve, i campi `importo_riserva` e `data_riserva` saranno `NULL`.

### Esecuzione SQL query di collegamento tabelle



In [ ]:
pd.read_sql_query(
'''
SELECT
    p.numero_polizza,
    p.prodotto,
    p.data_decorrenza,
    p.data_scadenza,
    r.importo_riserva,
    r.data_riserva
FROM
    polizze p
LEFT JOIN
    riserve r ON p.numero_polizza = r.numero_polizza;
''', conn)

,numero_polizza,prodotto,data_decorrenza,data_scadenza,importo_riserva,data_riserva
0,1000,MISTA,2020-02-23,2028-02-21,35870.33,2024-12-30
1,1001,CAPITALE,2020-02-03,2028-02-01,29499.91,2024-12-30
2,1002,MISTA,2020-02-21,2028-02-19,36560.92,2024-12-30
3,1003,MISTA,2020-03-02,2028-02-29,94743.68,2024-12-30
4,1004,MISTA,2020-03-15,2028-03-13,63741.04,2024-12-30
...,...,...,...,...,...,...
95,1095,CAPITALE,2020-02-28,2028-02-26,34487.34,2024-12-30
96,1096,CAPITALE,2020-02-12,2028-02-10,62433.27,2024-12-30
97,1097,CAPITALE,2020-03-10,2028-03-08,5079.09,2024-12-30
98,1098,MISTA,2020-01-18,2028-01-16,17222.19,2024-12-30


# Prompt Engineering per il debugging

In [ ]:
pd.read_sql_query(
'''
SELECT p.prodotto, SUM(r.importo_riserva) as riserva, r.data_riserva
FROM polizze
INNER JOIN riserve ON p.numero_polizza = r.numero_polizza
GROUP BY p.prodotto,r.data_riserva
''', conn)

DatabaseError: Execution failed on sql '
SELECT p.prodotto, SUM(r.importo_riserva) as riserva, r.data_riserva
FROM polizze
INNER JOIN riserve ON p.numero_polizza = r.numero_polizza
GROUP BY p.prodotto,r.data_riserva
': no such column: p.prodotto

### Utilizzo di LangChain e GPT-4o-mini per risolvere un problema nell'esecuzione di una query

In [ ]:
# Definizione del prompt
prompt = PromptTemplate(
    input_variables=["descrizione"],
    template="""
Sei un esperto analista in SQL.
Fornisci la query corretta in SQL per collegare le due tabelle con raggruppamento di dati e risolvendo il
problema illustrato nella descrizione fornita.

Tabelle:
1. polizze (numero_polizza INTEGER PRIMARY KEY, prodotto TEXT, data_decorrenza DATE, data_scadenza DATE)
2. riserve (numero_polizza INTEGER PRIMARY KEY, importo_riserva NUMERIC, data_riserva DATE)

Regole:
{descrizione}

SQL Query:
"""
)

chat = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
chain = prompt | chat

regole = """
Ho eseguito la seguente query per effettuare un raggruppamento della riserva per prodotto:
""
pd.read_sql_query(
'''
SELECT p.prodotto, SUM(r.importo_riserva) as riserva, r.data_riserva
FROM polizze
INNER JOIN riserve ON p.numero_polizza = r.numero_polizza
GROUP BY p.prodotto,r.data_riserva
''', conn)
""
ho il seguente messaggio di errore:
""
DatabaseError: Execution failed on sql '
SELECT p.prodotto, SUM(r.importo_riserva) as riserva, r.data_riserva
FROM polizze
INNER JOIN riserve ON p.numero_polizza = r.numero_polizza
GROUP BY p.prodotto,r.data_riserva
': no such column: p.prodotto
""
L'obiettivo è recuperare i dati dalle tabelle e fare un raggruppamento per prodotto sul totale delle riserve.
I campi da vedere sono prodotto, riserva e data riserva"""

sql_query = chain.invoke({"descrizione": regole})

display(Markdown(sql_query.content))

L'errore che stai riscontrando è dovuto al fatto che nella tua query SQL stai utilizzando l'alias `p` per la tabella `polizze`, ma non lo hai definito correttamente. Dovresti specificare l'alias `p` nella clausola `FROM`. Ecco la query corretta:

```sql
SELECT p.prodotto, SUM(r.importo_riserva) AS riserva
FROM polizze p
INNER JOIN riserve r ON p.numero_polizza = r.numero_polizza
GROUP BY p.prodotto
```

In questa query:

- Ho corretto l'alias della tabella `polizze` aggiungendo `p` dopo il nome della tabella nella clausola `FROM`.
- Ho rimosso `r.data_riserva` dalla clausola `SELECT` e dalla clausola `GROUP BY`, poiché non ha senso raggruppare per data di riserva se stai cercando un totale per prodotto. Se desideri visualizzare la data della riserva, dovresti decidere come aggregarla (ad esempio, potresti voler mostrare la data della riserva più recente o la data della prima riserva).

Se desideri includere la data della riserva più recente, puoi utilizzare una funzione di aggregazione come `MAX`:

```sql
SELECT p.prodotto, SUM(r.importo_riserva) AS riserva, MAX(r.data_riserva) AS data_riserva
FROM polizze p
INNER JOIN riserve r ON p.numero_polizza = r.numero_polizza
GROUP BY p.prodotto
```

Questa query ti darà il totale delle riserve per ogni prodotto e la data della riserva più recente associata a ciascun prodotto.

### Esecuzione SQL query di collegamento tabelle e raggruppamento

In [ ]:
pd.read_sql_query(
'''
SELECT polizze.prodotto, SUM(riserve.importo_riserva) AS riserva, MAX(riserve.data_riserva) AS data_riserva
FROM polizze
INNER JOIN riserve ON polizze.numero_polizza = riserve.numero_polizza
GROUP BY polizze.prodotto
''', conn)

,prodotto,riserva,data_riserva
0,CAPITALE,2636216.29,2024-12-30
1,MISTA,2448529.81,2024-12-30


# Prompt Engineering per la documentazione

### Utilizzo di LangChain e GPT-4o-mini per documentare il codice

In [ ]:
# Definizione del prompt
prompt = PromptTemplate(
    input_variables=["descrizione"],
    template="""
Sei un esperto analista in SQL. L'obiettivo è scrivere la documentazione di una query SQL.

Tabelle:
1. polizze (numero_polizza INTEGER PRIMARY KEY, prodotto TEXT, data_decorrenza DATE, data_scadenza DATE)
2. riserve (numero_polizza INTEGER PRIMARY KEY, importo_riserva NUMERIC, data_riserva DATE)

Regole:
{descrizione}

SQL Query:
"""
)

chat = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
chain = prompt | chat
regole = """
Ti chiedo di documentare linea per linea la seguente query scritta in SQL:
""
cursor.execute('''
DROP TABLE IF EXISTS polizze;
''')
# Creazione tabella polizze
cursor.execute('''
CREATE TABLE polizze (
    numero_polizza INTEGER PRIMARY KEY,
    prodotto TEXT,
    data_decorrenza DATE,
    data_scadenza DATE
);
''')
cursor.execute('''
DROP TABLE IF EXISTS riserve;
''')
# Creazione tabella riserve
cursor.execute('''
CREATE TABLE riserve (
    numero_polizza INTEGER PRIMARY KEY,
    importo_riserva NUMERIC,
    data_riserva DATE
);
''')
conn.commit()
""
"""
sql_query = chain.invoke({"descrizione": regole})
display(Markdown(sql_query.content))

Ecco la documentazione dettagliata della query SQL fornita, linea per linea:

```sql
# Inizio della query per l'esecuzione di comandi SQL tramite un cursore
cursor.execute('''
```
- Questa riga indica l'inizio di un blocco di codice SQL che verrà eseguito tramite il cursore `cursor`. Il cursore è un oggetto che consente di eseguire comandi SQL e recuperare dati dal database.

```sql
DROP TABLE IF EXISTS polizze;
```
- Questo comando SQL elimina la tabella `polizze` se esiste già nel database. L'uso di `IF EXISTS` evita errori nel caso in cui la tabella non sia presente, consentendo una gestione più fluida delle operazioni di creazione delle tabelle.

```sql
''')
```
- Questa riga chiude il blocco di codice SQL per il comando `DROP TABLE`. La query è ora completa e verrà eseguita.

```sql
# Creazione tabella polizze
cursor.execute('''
```
- Qui inizia un nuovo blocco di codice SQL per la creazione della tabella `polizze`. Il commento sopra indica che stiamo per definire la struttura della tabella.

```sql
CREATE TABLE polizze (
```
- Questo comando SQL inizia la definizione di una nuova tabella chiamata `polizze`. La tabella conterrà informazioni relative alle polizze assicurative.

```sql
    numero_polizza INTEGER PRIMARY KEY,
```
- Questa riga definisce una colonna chiamata `numero_polizza`, che è di tipo `INTEGER` e funge da chiave primaria della tabella. La chiave primaria garantisce che ogni valore in questa colonna sia unico e non nullo, identificando in modo univoco ogni polizza.

```sql
    prodotto TEXT,
```
- Qui viene definita una colonna chiamata `prodotto`, di tipo `TEXT`, che conterrà informazioni sul tipo di prodotto assicurativo associato alla polizza.

```sql
    data_decorrenza DATE,
```
- Questa riga definisce una colonna chiamata `data_decorrenza`, di tipo `DATE`, che rappresenta la data di inizio della copertura assicurativa della polizza.

```sql
    data_scadenza DATE
```
- Qui viene definita una colonna chiamata `data_scadenza`, anch'essa di tipo `DATE`, che indica la data di scadenza della polizza.

```sql
);
```
- Questa riga chiude la definizione della tabella `polizze`. La tabella è ora completamente definita e pronta per essere creata nel database.

```sql
''')
```
- Questa riga chiude il blocco di codice SQL per il comando `CREATE TABLE`. La query è ora completa e verrà eseguita.

```sql
cursor.execute('''
```
- Inizia un nuovo blocco di codice SQL per l'esecuzione di ulteriori comandi tramite il cursore.

```sql
DROP TABLE IF EXISTS riserve;
```
- Questo comando SQL elimina la tabella `riserve` se esiste già nel database, simile al comando precedente per la tabella `polizze`. L'uso di `IF EXISTS` previene errori se la tabella non è presente.

```sql
''')
```
- Questa riga chiude il blocco di codice SQL per il comando `DROP TABLE` relativo alla tabella `riserve`.

```sql
# Creazione tabella riserve
cursor.execute('''
```
- Qui inizia un nuovo blocco di codice SQL per la creazione della tabella `riserve`. Il commento sopra indica che stiamo per definire la struttura della tabella.

```sql
CREATE TABLE riserve (
```
- Questo comando SQL inizia la definizione di una nuova tabella chiamata `riserve`, che conterrà informazioni relative alle riserve associate alle polizze.

```sql
    numero_polizza INTEGER PRIMARY KEY,
```
- Questa riga definisce una colonna chiamata `numero_polizza`, di tipo `INTEGER` e funge da chiave primaria della tabella `riserve`. Questa colonna deve corrispondere al `numero_polizza` della tabella `polizze`, creando una relazione tra le due tabelle.

```sql
    importo_riserva NUMERIC,
```
- Qui viene definita una colonna chiamata `importo_riserva`, di tipo `NUMERIC`, che conterrà l'importo della riserva associata alla polizza.

```sql
    data_riserva DATE
```
- Questa riga definisce una colonna chiamata `data_riserva`, di tipo `DATE`, che rappresenta la data in cui è stata registrata la riserva.

```sql
);
```
- Questa riga chiude la definizione della tabella `riserve`. La tabella è ora completamente definita e pronta per essere creata nel database.

```sql
''')
```
- Questa riga chiude il blocco di codice SQL per il comando `CREATE TABLE` relativo alla tabella `riserve`.

```sql
conn.commit()
```
- Questo comando SQL esegue il commit delle modifiche al database. In altre parole, rende permanenti le operazioni di creazione delle tabelle `polizze` e `riserve`, assicurando che le tabelle siano effettivamente create nel database.

In sintesi, questa serie di comandi SQL serve a eliminare eventuali tabelle esistenti con nomi specifici (`polizze` e `riserve`), a crearne di nuove con la struttura definita e a confermare le modifiche nel database.

# Prompt Engineering per la traduzione

### Utilizzo di LangChain e GPT-4o-mini per effetuare traduzione di codice da R a Python

In [ ]:
# Definizione del prompt
prompt = PromptTemplate(
    input_variables=["descrizione"],
    template="""
Sei un esperto actuarial data scientist. Converti il codice da R in Python, secondo la descrizione e il seguente dataframe.

Dataframe:
eta_assicurato da 18 a 79 anni
eta_veicolo da 0 a 9 anni
esposizione da 0 a 1 anno
premio segue una distribuzione gamma

Regole:
{descrizione}

SQL Query:
"""
)
chat = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
chain = prompt | chat
regole = """
Puoi tradurre il codice utilizzato per creare un dataframe con R in Python
Il codice impiegato è il seguente:
set.seed(123)
n_rows <- 3000
eta_assicurato <- sample(18:79, size = n_rows, replace = TRUE)
eta_veicolo <- sample(0:9, size = n_rows, replace = TRUE)
esposizione <- round(runif(n_rows, min = 0, max = 1), 2)
premio <- round(rgamma(n_rows, shape = 2, rate = 1/100), 2)

df <- data.frame(
  eta_assicurato = eta_assicurato,
  eta_veicolo = eta_veicolo,
  esposizione = esposizione,
  premio = premio
)

"""

sql_query = chain.invoke({"descrizione": regole})

display(Markdown(sql_query.content))

Ecco come puoi convertire il codice R fornito in Python utilizzando la libreria `pandas` per la manipolazione dei dati e `numpy` per le operazioni numeriche. Inoltre, utilizzeremo `scipy.stats` per generare i valori della distribuzione gamma.

### Codice Python

```python
import pandas as pd
import numpy as np
from scipy.stats import gamma

# Imposta il seme per la riproducibilità
np.random.seed(123)

# Numero di righe
n_rows = 3000

# Genera i dati
eta_assicurato = np.random.choice(range(18, 80), size=n_rows, replace=True)
eta_veicolo = np.random.choice(range(0, 10), size=n_rows, replace=True)
esposizione = np.round(np.random.uniform(0, 1, size=n_rows), 2)
premio = np.round(gamma.rvs(a=2, scale=100, size=n_rows), 2)

# Crea il DataFrame
df = pd.DataFrame({
    'eta_assicurato': eta_assicurato,
    'eta_veicolo': eta_veicolo,
    'esposizione': esposizione,
    'premio': premio
})

# Visualizza le prime righe del DataFrame
print(df.head())
```

### Spiegazione del Codice

1. **Importazione delle librerie**: Importiamo `pandas` per la creazione del DataFrame, `numpy` per le operazioni numeriche e `scipy.stats` per generare i valori della distribuzione gamma.

2. **Impostazione del seme**: Utilizziamo `np.random.seed(123)` per garantire che i risultati siano riproducibili.

3. **Generazione dei dati**:
   - `eta_assicurato`: Generiamo un campione casuale di età tra 18 e 79 anni.
   - `eta_veicolo`: Generiamo un campione casuale di età del veicolo tra 0 e 9 anni.
   - `esposizione`: Generiamo un campione casuale uniforme tra 0 e 1, arrotondando a due decimali.
   - `premio`: Generiamo un campione dalla distribuzione gamma con forma `a=2` e scala `scale=100`, arrotondando a due decimali.

4. **Creazione del DataFrame**: Creiamo un DataFrame utilizzando i dati generati.

5. **Visualizzazione**: Stampiamo le prime righe del DataFrame per verificarne il contenuto.

Puoi eseguire questo codice in un ambiente Python con le librerie necessarie installate per ottenere un DataFrame simile a quello creato in R.

### Esecuzione codice in Python per creazione dataset

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gamma

# Imposta riproducibilità
np.random.seed(123)

# Numero di righe
n_rows = 3000

# Genera i dati
eta_assicurato = np.random.choice(range(18, 80), size=n_rows, replace=True)
eta_veicolo = np.random.choice(range(0, 10), size=n_rows, replace=True)
esposizione = np.round(np.random.uniform(0, 1, size=n_rows), 2)
premio = np.round(gamma.rvs(a=2, scale=100, size=n_rows), 2)

# Crea il DataFrame
df = pd.DataFrame({
    'eta_assicurato': eta_assicurato,
    'eta_veicolo': eta_veicolo,
    'esposizione': esposizione,
    'premio': premio
})

# Visualizza le prime righe del DataFrame
print(df.head())


   eta_assicurato  eta_veicolo  esposizione  premio
0              63            8         0.91   62.57
1              20            7         0.35   29.48
2              46            0         0.61  489.75
3              52            3         0.19   70.91
4              56            3         0.04   76.60


In [ ]:
cursor.close()
conn.close()

### Riferimenti

In [ ]:
# https://platform.openai.com/docs/overview
# https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html
# https://python.langchain.com/docs/integrations/chat/openai/
# https://pypi.org/project/langchain-openai/